In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from datetime import date, timedelta
from dotenv import load_dotenv
import os
import time
from selenium.webdriver.common.keys import Keys

# carregando as variáveis de ambiente
load_dotenv()


True

In [128]:
def retornar_periodo():
    hoje = date.today()
    amanha = hoje + timedelta(days=1)
    hoje_str = date.strftime(hoje, '%d/%m/%Y')
    amanha_str = date.strftime(amanha, '%d/%m/%Y')
    periodo_str = (f'{hoje_str} - {amanha_str}')
    return periodo_str


In [136]:
def obter_driver():
    usuario = os.getlogin()
    chrome_options = Options()
    chrome_options.add_argument(
        f'user-data-dir=C:/Users/{usuario}/AppData/Local/Google/Chrome/Selenium'
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # maximixando a tela
    driver.maximize_window()
    return driver

In [135]:
def logar(driver):
    # indo para a tela de login
    driver.get("https://qualylab.gerencialab.com.br/")

    # aguardando elemento aparecer na tela
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div/div[1]/a/img'))
        )
    time.sleep(4)

    # obtendo as variáveis de ambiente
    login = os.getenv('LOGIN')
    password = os.getenv('PASSWORD')

    # preenchendo o campo usuário
    driver.find_element(By.XPATH, '//*[@id="loginsite"]').send_keys(login)

    # preenchendo o campo senha
    driver.find_element(By.XPATH, '//*[@id="senhasite"]').send_keys(password)

    # clicando em acessar
    driver.find_element(By.XPATH, '//*[@id="authLogin"]').click()

    # aguardando logo GerenciaLab Aparecer na tela
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[1]/div/a/img')))
    time.sleep(4)

    return True

In [134]:
# função que obtém a lista das amostras
def obter_dados(driver):
    # indo para ordens de serviço
    driver.get("https://qualylab.gerencialab.com.br/service-order")
    # aguardando label Ordem serviços aparecer na tela
    elemento = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-wrapper"]/div/div/h4')))
    time.sleep(4)
    # clicando no botão Limpar Pesquisa
    driver.find_element(By.XPATH, '//*[@id="ButtonsShowTable"]/div/button[5]').click()
    time.sleep(4)
    # obtendo o campo da data de entrega
    campo_data = driver.find_element(By.XPATH, '//*[@id="dataPrazoEntregaOSForm"]')
    campo_data.send_keys(retornar_periodo())

    # clicando em aplicar
    driver.find_element(By.XPATH, '/html/body/div[5]/div[4]/button[2]').click()
    time.sleep(4)

    # obtendo a lista de elementos
    lista_elementos = driver.find_elements(By.CSS_SELECTOR, '#tableOrdemdeServico tr.even, #tableOrdemdeServico tr.odd')

    amostras = []
    for linha in lista_elementos:
        cliente = linha.find_elements(By.TAG_NAME, 'td')[4].text
        status_os = linha.find_elements(By.TAG_NAME, 'td')[1].text
        amostra = linha.find_elements(By.TAG_NAME, 'td')[2].text
        prioridade = linha.find_elements(By.TAG_NAME, 'td')[3].text
        amostras.append((status_os, amostra, cliente, prioridade))

    return amostras

In [137]:
def sair_sistema(driver):
    driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/ul/li[4]/a').click()
    time.sleep(4)
    driver.close()
    

In [138]:
# testes
driver = obter_driver()
logado = logar(driver)
if logado:
    dados = obter_dados(driver)
    print(dados)
    sair_sistema(driver)
else:
    print('Não foi possível logar')

[('Aguardando agendamento', 'AM00000093/2025', 'Uanderson ( Uanderson )', 'Padrão'), ('Aguardando agendamento', 'AM00000076/2025', 'Waldemberg Pereira (\nWaldemberg Pereira )', 'Padrão'), ('Aguardando agendamento', 'AM00000074/2025', 'Waldemberg Pereira (\nWaldemberg Pereira )', 'Padrão'), ('Aguardando agendamento', 'AM00000075/2025', 'Waldemberg Pereira (\nWaldemberg Pereira )', 'Padrão'), ('Aguardando agendamento', 'AM00000068/2025', 'Uanderson ( Uanderson )', 'Padrão'), ('Aguardando agendamento', 'AM00000069/2025', 'Uanderson ( Uanderson )', 'Padrão')]
